In [1]:
# Magic + imports
%load_ext autoreload
%autoreload 2
%matplotlib inline

CMAP = "Blues"

In [2]:
# set path
import sys
sys.path.append('/home/jovyan/odc-hub/')
# import required modules
import warnings
warnings.filterwarnings('ignore')
import xarray as xr 
import functools as fct
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce 
import pandas as pd

# import required datacube modules
from utils_sac.interactive_maps import display_map
from utils_sac.notebook_animation_widgets import create_movie_on_np_array
from utils_sac.plotting import plot_availability

import utils_sac.waterline_functions_deaafrica as waterline_funcs
from utils_sac.clean_mask import landsat_qa_clean_mask
from utils_dcal.data_cube_utilities.dc_water_classifier import wofs_classify

import datacube
dc = datacube.Datacube(app='wofs dask')

In [3]:
import dask
from dask.distributed import Client
client = Client('dask-scheduler.dask.svc.cluster.local:8786')

In [4]:
lon_range = (177.481, 177.855)
lat_range = (-18.00, -18.3732)
#smaller area
#lon_range = (177.36740112307, 177.46044158938)
#lat_range = (-18.107528686523, -18.159027099609)
time_range = ("2017")
tide_range = (0.00, 10.00)
time_step = '1Y'
output_projection = "EPSG:32760"

path = "/home/shared/geo_demos/tides/"
tide_file_2017 = "/home/shared/geo_demos/tides/IDO70004_2017.csv"
tide_file_2018 = "/home/shared/geo_demos/tides/IDO70004_2018.csv"


In [5]:
waterline_funcs.display_map(latitude=lat_range, longitude=lon_range)

In [6]:
# Create the 'query' dictionary object, which contains the longitudes, latitudes and time provided above
query = {
    'y': lat_range,
    'x': lon_range,
    'time': time_range,
    'output_crs': output_projection,  
    'resolution': (-90, 90)
}

In [ ]:
#landsat_ds = dc.load(product="ls8_usgs_wofs_scene",
#               group_by='solar_day',
#               #measurements = ["green","red", "blue", "nir","swir1","swir2","coastal_aerosol","pixel_qa"],
#               **query)

In [9]:
#use raw S2 and apply wofs function from utils.
scenes = dc.load(product="ls8_usgs_sr_scene",
               group_by='solar_day',
               measurements = ["green","red", "blue", "nir","swir1","swir2","coastal_aerosol","pixel_qa"],
               **query)

In [8]:
#apply wofs classifier from utils. 
clearsky_masks = landsat_qa_clean_mask(scenes, 'LANDSAT_8')
clearsky_scenes = scenes.where(clearsky_masks)
water_classes = wofs_classify(scenes, clean_mask=clearsky_masks.values , no_data = np.nan , x_coord='x', y_coord = "y")
 

AttributeError: 'Dataset' object has no attribute 'pixel_qa'

In [ ]:
#landsat_ds.water.isel(time=3).plot(cmap='RdBu', size=6, vmin=-0.5, vmax=0.5)
water_classes.isel(time=3).plot(cmap='RdBu', size=6, vmin=-0.5, vmax=0.5)
#plt.show()

In [ ]:
tide_data_2017 = pd.read_csv(tide_file_2017, parse_dates=['time'], index_col='time')
tide_data_2018 = pd.read_csv(tide_file_2018, parse_dates=['time'], index_col='time')
tide_data = pd.concat([tide_data_2017, tide_data_2018])
tide_data.head()
tide_data['tide_height'] = tide_data['tide']
tide_data.head()

In [ ]:
# First, we convert the data to an xarray dataset so we can analyse it in the same way as our Landsat data
tide_data_xr = tide_data.to_xarray()

# We want to convert our hourly tide heights to estimates of exactly how high the tide was at the time that
# each satellite image was taken. To do this, we can use `.interp` to 'interpolate' a tide height for each
# Landsat timestamp:
#landsat_tideheights = tide_data_xr.interp(time=landsat_ds.time)

landsat_tideheights = tide_data_xr.interp(time=water_classes.time)

# We then want to put these values back into the Landsat dataset so that each image has an estimated tide height:
landsat_ds['tide_height'] = landsat_tideheights.tide_height
#clearsky_scenes['tide_height'] = landsat_tideheights.tide_height


# Plot the resulting tide heights for each Landsat image:
landsat_ds.tide_height.plot()
#clearsky_scenes.tide_height.plot()
plt.show()

In [ ]:
landsat_hightide = landsat_ds.where((landsat_ds.tide_height > tide_range[0]) & 
                                    (landsat_ds.tide_height < tide_range[1]), drop=True)
#landsat_hightide = landsat_ds.where((landsat_ds == 1), drop=True)
landsat_hightide = landsat_ds.where(landsat_ds.water != 1)  
#landsat_hightide = clearsky_scenes.where((clearsky_scenes.tide_height > tide_range[0]) & 
 #                                  (clearsky_scenes.tide_height < tide_range[1]), drop=True)
print(landsat_hightide)
landsat_hightide.water.isel(time=3).plot(cmap='YlOrRd', size=6, vmin=-0.5, vmax=0.5)

In [ ]:
landsat_resampled = landsat_hightide.water.compute().resample(time=time_step).min('time')
landsat_resampled.plot(col='time', cmap='RdBu', col_wrap=3, vmin=-0.5, vmax=0.5)

In [ ]:
# Set up attributes to assign to each waterline
attribute_data = {'time': [str(i)[0:10] for i in landsat_resampled.time.values]}
attribute_dtypes = {'time': 'str'}

# Extract waterline contours for the '0' water index threshold:
contour_gdf = waterline_funcs.contour_extract(
    z_values=[0],
    ds_array=landsat_resampled,
    ds_crs=landsat_ds.crs,
    ds_affine=landsat_ds.geobox.transform,
    output_shp=f'output_waterlines.shp',
    attribute_data=attribute_data,
    attribute_dtypes=attribute_dtypes,
    min_vertices=50
)

# Plot output shapefile over the top of the first year's MNDWI layer
fig, ax = plt.subplots(1, 1, figsize=(20, 7))
#landsat_resampled.isel(time=-1).plot(ax=ax, cmap='Greys', alpha=1.0, edgecolors=None)
contour_gdf.plot(cmap='YlOrRd', ax=ax)


In [ ]:
waterline_funcs.map_shapefile(gdf=contour_gdf)

In [ ]:
client.restart()